In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [15]:
pd.set_option('display.max_colwidth', 10)
pd.set_option('display.max_columns', None)

In [16]:
'''Read dataset'''
cdf_applicant = pd.read_csv('data/cdf_applicant.csv', index_col=['ApplicantID'])
cdf_applicant_experience = pd.read_csv('data/cdf_applicant_experience.csv', index_col=['ApplicantID'])
cdf_stage = pd.read_csv('data/cdf_stage.csv', index_col=['StageID'])
cdf_pipeline = pd.read_csv('data/cdf_pipeline.csv', index_col=['PipelineID'])

'''Merge dataset'''
df_merged = pd.merge(
    cdf_applicant, cdf_applicant_experience, on=['ApplicantID']
)

df_merged = pd.merge(
    df_merged, cdf_pipeline, on=['ApplicantID']
)

df_merged = pd.merge(
    df_merged, cdf_stage, on=['StageID']
)

df_merged.drop(columns=['StageID'], inplace=True)

df_merged.set_index(['ApplicantID'], inplace=True)

'''Hal yang terlewat'''
df_merged.JobDescription.fillna('', inplace=True)
df_merged.drop(index=13, inplace=True)


In [20]:
cdf_job = pd.read_csv('data/cdf_job.csv')

In [22]:
cdf_job

,JobID,Description,EducationLevelID,FunctionPositionID,JobStatus,JobTitle,Requirement,SalaryMax,SalaryMin,HiredQuota,DepartmentID,CityID,CompanyID,ProvinceID,SkipTest,MajorID,DbName,UsiaMax,UsingGlasses,IQMin,IsSpecificEducationLevel,DriverLicenseType,IsSpesificDriverLicense,Gender,IsSpecificAge,IsSpecificGender,IsSpecificIQ,IsSpecificMarital,MaritalStatus,AdsStatisticID
0,1,under ...,4,1,Close,night ...,with y...,3600000,3300000,1,1,1,7,1,True,1,WarnaW...,0.0,False,0,False,0,False,0,False,False,False,False,0,0
1,2,becomi...,2,2,Close,book k...,bachel...,4250000,3500000,1,2,1,7,1,True,2,WarnaW...,0.0,False,0,False,0,False,0,False,False,False,False,0,0
2,3,develo...,2,3,Close,it and...,usia m...,5000000,4000000,2,8,1,1,1,False,4,WarnaW...,0.0,False,0,False,0,False,0,False,False,False,False,0,0
3,4,melaku...,2,4,Close,sekret...,pendid...,4500000,3600000,1,8,1,1,1,False,6,WarnaW...,0.0,False,0,False,0,False,0,False,False,False,False,0,0
4,5,bantu ...,4,5,Close,sekret...,usia m...,5000000,4000000,3,8,1,1,1,True,7,WarnaW...,0.0,False,0,False,0,False,0,False,False,False,False,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921,3090,memimp...,9,46,Publish,genera...,usia m...,12000000,8000000,1,13,1,3,1,True,7,WarnaW...,0.0,False,0,False,0,False,0,False,False,False,False,0,0
922,3091,NaN,9,6,Publish,market...,NaN,4300000,4000000,1,10,1,1,1,True,7,WarnaW...,NaN,0,0,False,0,False,0,False,False,False,False,0,0
923,3092,mengua...,2,39,Publish,manage...,usia m...,0,0,1,49,2,13,2,True,2,WarnaW...,40.0,False,100,True,0,False,0,True,False,True,False,0,0
924,3093,menger...,9,26,Publish,genera...,pria w...,5000000,4300000,1,13,1,3,1,True,15,WarnaW...,0.0,False,0,False,0,False,0,False,False,False,False,0,0


In [18]:
df_merged[df_merged.Label == 'Hired']

,DiseaseHistory,Age,DriverLicenseType,Gender,IsUsingGlasses,MaritalStatus,Nationality,Strengthness,Weaknesses,TypeOfVehicle,Height,ExpectedSalary,CityID,ProvinceID,Industry,CompanyName,JobDescription,Position,Salary,YearsOfExperience,JobID,Label
ApplicantID,,,,,,,,,,,,,,,,,,,,,,
67,0,30,c,female,true,single,indonesia,disiplin,terbur...,motor,152,4000000,1,1,pelaya...,lembag...,memban...,staff ...,0,1,105,Hired
128,amandel,0,c,male,false,single,indonesia,negosi...,bahasa...,motor,176,3600000,1,1,jasa g...,pt dya...,mempro...,freela...,2750000,3,3006,Hired
134,0,30,c,female,true,single,indonesia,respon...,having...,motor,156,0,1,1,law se...,sidabu...,respon...,legal ...,0,2,4,Hired
457,0,28,c,male,true,single,indonesia,kreati...,cerobo...,motor,170,4500000,1,1,teleko...,pt tel...,menjag...,networ...,0,0,8,Hired
531,0,30,c,female,true,single,indonesia,respon...,having...,motor,156,4500000,1,1,law se...,kantor...,respon...,legal ...,3300000,2,26,Hired
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39997,0,57,c,male,true,married,indonesia,instal...,menggu...,motor,170,4500000,1,1,instal...,pt fuj...,mengaw...,superv...,6300000,3,1641,Hired
40043,0,31,c,male,false,married,indonesia,memili...,menyuk...,motor,170,6000000,0,1,erp pr...,pt gbs...,saya b...,progra...,5500000,6,1618,Hired
43028,0,48,a,male,false,married,indonesia,0,0,0,0,0,75,3,outsou...,pt sum...,sebaga...,driver,0,2,2799,Hired
